In [1]:
import json
import os
from pathlib import Path
import yaml
from tqdm import tqdm

annotation_file = "instances_val2017.json"
classes_file = "classes.yaml"
colors_file = "colors.yaml"

os.makedirs(Path.cwd().joinpath("cocodataset/detection-results"), exist_ok=True)
os.makedirs(Path.cwd().joinpath("cocodataset/ground-truth"), exist_ok=True)
os.makedirs(Path.cwd().joinpath("cocodataset/images"), exist_ok=True)

annotation_file_path = Path.cwd().joinpath("input/annotations").joinpath(annotation_file)
classes_file_path = Path.cwd().joinpath("input/classes").joinpath(classes_file)
colors_file_path = Path.cwd().joinpath("input/classes").joinpath(colors_file)

with open(annotation_file_path, "r") as f:
    data = json.load(f)

with open(classes_file_path, "r") as f:
    classes = yaml.safe_load(f)

with open(colors_file_path, "r") as f:
    colors = yaml.safe_load(f)

## downloading and saving images

In [2]:
import requests
import io
from PIL import Image

def fetch_and_save(url, filename, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, filename)

    response = requests.get(url)
    if response.status_code == 200:
        image_content = io.BytesIO(response.content)
        pil_image = Image.open(image_content).convert("RGB")
        pil_image.save(filepath)
    else:
        print(f"filename: {filename} from {url} cannot be downloaded")
    


In [2]:
filenames = [x["file_name"] for x in data["images"]]
urls = [x["coco_url"] for x in data["images"]]
ids = [x["id"] for x in data["images"]]

annotations = [(x["bbox"], x["category_id"], x["image_id"]) for x in data["annotations"]] # bbox, category_id, image_id

In [ ]:
pbar = tqdm(total=len(urls))
for url, filename in zip(urls, filenames):
    pbar.update()
    fetch_and_save(url, filename, output_dir=Path.cwd().joinpath("cocodataset/images"))

## Make ground truth files

In [ ]:
for i in tqdm(range(len(ids))):
    text = ""
    for ann in annotations:
        if ann[-1] == ids[i]:
            bbox = ann[0]
            id = ann[1]
            if id > 80:
                continue
            bbox = list(map(round, bbox))
            x1, y1 = bbox[0], bbox[1]
            x2, y2 = x1 + bbox[2], y1 + bbox[3]
            line = ""
            line += str(classes[id]).replace(" ", "")
            line += " " + " ".join(map(str, [x1, y1, x2, y2])) + "\n"
            text += line

    filename = filenames[i].split(".")[0]+".txt"
    filepath = Path.cwd().joinpath("cocodataset/ground-truth").joinpath(filename).__str__()
    with open(filepath, "w") as f:
        f.write(text)
    


In [ ]:
from glob import glob
len(glob(os.path.join("cocodataset/images", "*.jpg")))

In [ ]:

from mAP import compute_metrics, compute_mAP

ground_truth_files = glob(os.path.join("cocodataset/ground-truth", "*.txt"))
ground_truth_files = list(map(os.path.basename, ground_truth_files))
ground_truth_files.sort()

detection_result_files = glob(os.path.join("cocodataset/detection-results", "*.txt"))
detection_result_files = list(map(os.path.basename, detection_result_files))
detection_result_files.sort()

assert ground_truth_files == detection_result_files

ground_truth, detection_result = [], []

for gt_f, dr_f in zip(ground_truth_files, detection_result_files):
    ground_truth_file, detection_result_file = os.path.join("cocodataset/ground-truth", gt_f), os.path.join("cocodataset/detection-results", dr_f)

    with open(ground_truth_file, "r") as f:
        ground_truth_lines = f.read()

    with open(detection_result_file, "r") as f:
        detection_result_lines = f.read()

    ground_truth_data = []

    for line in ground_truth_lines:
        data = line.split(" ")
        data = map(lambda class_name, x1, y1, x2, y2: (str(class_name), int(x1), int(y1), int(x2), int(y2)), data)
        ground_truth_data.append(data)

    detection_result_data = []

    for line in detection_result_lines:
        data = line.split(" ")
        data = map(lambda class_name, score, x1, y1, x2, y2: (str(class_name), float(score), int(x1), int(y1), int(x2), int(y2)), data)
        detection_result_data.append(data)

    ground_truth.append(ground_truth_data)
    detection_result.append(detection_result_data)


assert len(ground_truth) == len(detection_result)

AP = compute_metrics(ground_truth, detection_result)

mAP = compute_mAP(AP)

In [ ]:
from mAP import compute_metrics, compute_mAP
AP = compute_metrics([[["glass", 0.60, 1, 5, 20, 100], ["glass", 0.60, 1, 5, 20, 100]]], [[["glass", 1, 5, 20, 100], ["glass", 1, 5, 20, 100]]])
print(AP)